# OPTIMIZACIÓN DE HIPERPARÁMETROS DE UNA RED NEURONAL MEDIANTE UN ALGORITMO GENÉTICO

En este notebook procedo a hacer una optimización de hiperparámetros de una red neuronal mediante un algoritmo genético (GA) para la solución de un problema de regresión lineal.


Explicando un poco de teoría, decir a grandes rasgos que un algoritmo genético es un algoritmo de optimización de hiperparámetros bioinspirado, es decir, inspirado en la biología y sus procesos. En este caso, el algoritmo genético trata de optimizar hiperparámetros fijándose en como las poblaciones en un entorno natural evolucionan a lo largo del tiempo, siendo los individuos más adaptados a su entorno o circunstancias los que más éxito reproductivo tienen, concepto conocido como fitness, y por tanto son los que en mayor medida dejan descendencia. A su vez, la descendencia en su concepción puede mutar, pudiendo, en un proceso totalmente aleatorio, mejorar el fitness de sus padres, mantenerlo o empeorarlo. Por consiguiente, concretando, en este algoritmo se parte de una población de individuos, que serán en esencia distintas combinaciones de hiperparámetros (cada hiperparámetro será considerado como un gen). Estas combinaciones de hiperparámetros serán evaluadas en función del problema a resolver y el resultado en una métrica asociada a este, que será el fitness. Los individuos que mejor fitness obtengan serán seleccionados para mezclarse entre ellos y pasar sus genes a los hijos de la siguiente generación de la población, que estará constituida por hijos y padres. A su vez los hijos tendrán cierta probabilidad de mutar, traduciéndose esto en un cambio aleatorio del valor de un hiperparámetro por otro (lo que permitirá añadir espontáneamente otros valores que quizás mejoren el fitness). Esto irá produciéndose recursivamente a lo largo de un número fijo de iteraciones o generaciones, determinado por el usuario o programador.

***Planteando más detalladamente el problema a resolver***, este consiste en hallar los mejores hiperparámetros para una red neuronal que mejor predicción permitan realizar para la variable respuesta MEDV (Median value of owner-occupied homes in $\1000's) en el famoso dataset de datos de la vivienda en Boston. La métrica a usar como ***fitness*** será la función de pérdida ***mean squared logarithmic error*** (cuanto menor sea el valor, mejor el resultado). ***Los hiperparámetros a mejorar*** serán las funciones de activación para las capas desde la de entrada hasta la última de las ocultas, el nº de  capas de neuronas y nº de neuronas de dichas capas (considerados ambos hiperparámetros en el planteamiento del algoritmo como un solo gen), el optimizador, el número de épocas (epochs) y el hiperparámetro batch_size.

Importo las librerías a usar:

In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
import random
from sklearn.model_selection import train_test_split


El dataset ha sido extraído de la librería de python Scikit Learn:

In [2]:
from sklearn.datasets import load_boston

boston = load_boston() #Cargo el dataset

boston_df = pd.DataFrame(boston.data, columns=boston.feature_names) #Paso a dataframe sus variables explicativas.
boston_df['MEDV'] = boston.target # Y le añado la variable respuesta.

In [3]:
boston_df #Aquí se puede ver el dataset en formato dataframe.

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


Para el proceso de evaluación de la red neuronal, declaro en variables cuál va a ser el conjunto de variables explicativas (x), y cuál va a ser la variable respuesta:

In [4]:
x = boston_df.drop(['MEDV'], axis=1)
y=boston_df['MEDV']

Posteriormente divido entre train y test, paso necesario para cualquier proceso de Machine Learning supervisado:

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=1)

Como la red neuronal no admite las etiquetas de los campos del dataframe, me quedo únicamente con los valores "crudos":

In [6]:
x_train = x_train.values
y_train = y_train.values
x_test = x_test.values
y_test = y_test.values

Defino una función que se encargará de ejecutar la red neuronal, que será ejecutada para evaluar a los distintos individuos de la población. Correré la red neuronal para cada individuo o combinación de hiperparámetros:

In [7]:


def neural_model(activation, layers, optimizer):

    model = Sequential()
    
    for i, neuron in enumerate(layers):  # Nº de capas, desde la capa de entrada hasta la última de 
                                         # las capas ocultas.
        if i == 0:
            model.add(Dense(neuron, input_dim=13, activation=activation)) # las funciones de activación desde
                                                                          # la capa de entrada hasta la capa
                                                                          # de salida.
        else:
            model.add(Dense(neuron, activation=activation))
    
    model.add(Dense(1, activation='linear')) # Al ser un proceso de regresión linear, está claro que la función
                                             # de la capa de salida será lineal.
    
    model.compile(loss='mean_squared_logarithmic_error', optimizer=optimizer, metrics=['mse']) #Aquí se especifica
                                                                                               #la métrica a usar
                                                                                               # para el fitness.   
    
    return model   

Creo una función para crear la población inicial, con todos los hiperparámetros a optimizar como genes de cada individuo de esta población. Esta función tiene un parámetro de entrada que será el número de individuos que se desea crear:

In [8]:
activation = ['relu', 'softmax', 'tanh'] # Defino una lista que tendrá las posibles funciones de activación
                                         # para cada individuo. Esta lista se usará para construir individuos
                                         # en la función que creará la población inicial.


optimizer = ['adam','sgd', 'adamax'] # Defino una lista que tendrá los posibles optimizadores para cada
                                     # individuo.Esta lista se usará para construir individuos
                                     # en la función que creará la población inicial.

def create_population(individuals_num):
    
    population = [[random.choice(activation),random.sample(range(1,40), random.randint(1,5)), 
                   random.choice(optimizer), random.randint(20,400), random.randint(10,150)] 
                  for i in range(individuals_num)] 
    
    return population


Defino una función que asignará el fitness o éxito reproductivo de cada individuo de la población en función de su desempeño tras entrenar con el conjunto de entrenamiento y evaluar con el conjunto de test la red neuronal con sus hiperparámetros:

In [9]:
def fitness_eval(individual):
    
    model = neural_model(individual[0],individual[1],individual[2])
    
    model.fit(x_train, y_train, epochs=individual[3],batch_size=individual[4], verbose=0)#Aquí los hiperparámetros
                                                                                        #de epochs y batch_size.
    
    evaluate = model.evaluate(x_test, y_test, verbose=0) #Tras entrenar evaluamos con test. El fitness aquí
                                                         #obtenido se medirá con la métrica mean squared 
                                                        #logarithmic error (evaluate[0]).
    
    return evaluate[0]
    

Función que seleccionará a los padres de la siguiente generación. Esta función seleccionará n padres con los mejores fitness de la población:

In [10]:
def parents_selection(population,n_parents, fitness):
    
    parents = []
    
    sorting_by_fitness_list = [[i,j] for i,j in zip(fitness, new_population)] #Creo una lista en la que aparecerán
                                                                              #los individuos con su fitness
                                                                              #correspondiente.
                                                                                                          
    sorting_by_fitness_list.sort(key=lambda x:x[0]) #Posteriormente ordeno de mejor fitness ('menor valor de
                                                     #mean squared logarithmic error') a peor ('mayor valor de
                                                     #mean suared logarithmic error').
    
    for parent in range(n_parents): # Hecho lo anterior selecciono los n padres con mejor fitness de la población.
        
        parents.append(sorting_by_fitness_list[parent][1])       
        
    return parents 

Creo una función con la que generar a los hijos de los padres seleccionados. Mencionar que he decidido que el punto de cruce (crosspoint) sea el elemento/gen en la posición 2 para que más o menos cada padre aporte el hijo aproximadamente la mitad de su información, siendo así más "bioinspirado" :

In [11]:
def cross(n_children, parents):
    
    children = [] # Esta lista almacenará a los hijos generados por el cruce de los padres
    cross_records =[]#Esta lista almacenará la combinación de padres reproductores para consultarla posteriormente
                    #y no repetir cruces idénticos.
    
    for i in range(n_children):
        
        crosspoint = 2  
        
        random_ids = random.sample(range(5), 2)
        
        #No es deseable ni que un individuo se cruce consigo mismo ni que se hagan dos veces los mismos cruces:
        while random_ids[0] == random_ids[1] or random_ids in cross_records:
            
            random_ids = random.sample(range(5), 2)
        
        cross_records.append(random_ids)
            
        reproductive_parent_1 = parents[random_ids[0]]
        reproductive_parent_2 = parents[random_ids[1]]
        
        children.append(reproductive_parent_1[0:crosspoint+1] + reproductive_parent_2[crosspoint+1:5])
        
        
    return children

Creo una función que mutará con cierta probabilidad a los hijos o individuos generados del cruce de los padres:

In [12]:
def mutation(children, n_children, mutation_rate): #Hay que establecer cierto ratio o probabilidad de mutación
                                                   #(mutation_rate). Por lo general no debe ser alta.
    
    
    for i in range(n_children):
        
        if random.random()<= mutation_rate:  #Cada individuo tiene una probabilidad de mutar (mutation_rate)
            
            mutation_point = random.randint(0, 4) # Se selecciona aleatoriamente el gen que va a ser mutado
                                                  # en el individuo
            
            
            # Se trata la mutación de los hiperparámetros en función de sus particularidades:
            
            if mutation_point == 0:
                new_value =random.choice(activation)
                while new_value == children[i][mutation_point]:
                    new_value =random.choice(activation)
                    
            
            elif mutation_point == 1:
                new_value = random.sample(range(1,40), random.randint(1,5))
                while new_value == children[i][mutation_point]:
                    new_value = random.sample(range(1,40), random.randint(1,5))
                
            
            elif mutation_point == 2:
                new_value = random.choice(optimizer)
                while new_value == children[i][mutation_point]:
                    new_value = random.choice(optimizer)
                    
            
            elif mutation_point == 3:
                new_value = random.randint(20,400)
                while new_value == children[i][mutation_point]:
                    new_value = random.randint(20,400)
                    
            
            elif mutation_point == 4:
                new_value = random.randint(10,150)
                while new_value == children[i][mutation_point]:
                    new_value = random.randint(10,150)
                    
        
            
            children[i][mutation_point] = new_value
            
            
    return children              

Creo también una función que guardará en forma de diccionario los resultados de las distintas generaciones en la ejecución del algoritmo genético. El diccionario resultante permitirá crear un historial para revisar los diferentes resultados y distintas combinaciones de hiperparámetros, u observar los distintos fitness para  buscar la mejor métrica y por tanto la mejor solución, que no tiene por qué ser la última arrojada por el proceso de optimización:

In [13]:
def gen_dic(dic, generation, fitness, population):
    
    if dic == {}:
        dic['generation'] = list()
        dic['population'] = list()
        dic['fitness_pop'] = list()
        dic['best_fitness'] = list()

    
    

    dic['generation'].append(generation)
    dic['population'].append(population)
    dic['fitness_pop'].append(fitness)
    dic['best_fitness'].append(min(fitness))

Finalmente me dispongo a ejecutar el algoritmo genético. Pero antes establezco el valor de algunos parámetros necesarios para estas funciones:

In [14]:
individuals_num = 10 #Nº de individuos de la población.
generation_num = 20 #Nº de generaciones en las que la solución va a irse optimizando.
mutation_rate = 0.2 # Probabilidad que tiene cada individuo de la población de mutar.
n_parents = 5 #Nº de padres que queremos seleccionar.
n_children = 5 #Nº de hijos que se generarán.

Por último, procedo a ejecutar las funciones que correrán el algoritmo:

In [15]:
new_population = create_population(individuals_num)

genetic_dict = {}

num = 3

for generation in range(generation_num):
    
    print(f'GENERATION {generation} \n') #Con estos "prints" se podrá ver cómo va progresando el algoritmo
    print(new_population, '\n')
    
    fitness = [fitness_eval(i) for i in new_population] #Generará una lista con los distintos fitness de
                                                        #cada individuo de la población actual.
    print(f'fitness: {fitness} \n') #Este print mostrará en pantalla los fitness.
    print(f'fitness medio de la población: {sum(fitness)/individuals_num} \n') 
    print(f'mejor fitness:  {min(fitness)} \n')
    
    gen_dic(genetic_dict, generation, fitness, new_population)
                                                                                                            
    parents = parents_selection(new_population, n_parents, fitness)
    children = cross(n_children, parents)
    children_with_mutations = mutation(children, n_children, mutation_rate)
    
    
    
    #new_population[0:parents.shape[0], :] = parents
    #new_population[parents.shape[0]:, :] = children_with_mutations
    new_population = parents + children_with_mutations #La nueva población será una mezcla de los padres 
                                                       #(individuos de la generación anterior con mejor
                                                        # fitness. Esto ayudará a que persistan en las
                                                         # generaciones soluciones buenas) y los hijos mutados,
                                                         # producto del cruce de los individuos con mejor
                                                         # fitness de la generación anterior (individuos
                                                          # que generan a priori mejores soluciones
                                                           # para el problema a optimizar).

GENERATION 0 

[['tanh', [10], 'adamax', 66, 69], ['tanh', [31, 16, 37, 32, 35], 'sgd', 88, 56], ['relu', [21, 6], 'adam', 380, 147], ['tanh', [29], 'adamax', 239, 51], ['relu', [11], 'sgd', 34, 40], ['tanh', [17, 8, 12, 28], 'adamax', 392, 11], ['softmax', [30, 32, 10], 'adam', 148, 22], ['tanh', [36], 'adamax', 94, 95], ['relu', [32], 'adamax', 28, 98], ['tanh', [9, 38, 24, 17, 8], 'adamax', 75, 40]] 

fitness: [9.641816139221191, 0.17879626154899597, 9.641816139221191, 0.4272805154323578, 0.12618108093738556, 0.09680227935314178, 1.8206522464752197, 0.512104868888855, 0.2345270961523056, 9.641816139221191] 

fitness medio de la población: 3.2321792766451836 

mejor fitness:  0.09680227935314178 

GENERATION 1 

[['tanh', [17, 8, 12, 28], 'adamax', 392, 11], ['relu', [11], 'sgd', 34, 40], ['tanh', [31, 16, 37, 32, 35], 'sgd', 88, 56], ['relu', [32], 'adamax', 28, 98], ['tanh', [29], 'adamax', 239, 51], ['tanh', [29], 'adamax', 392, 11], ['tanh', [29], 'adamax', 34, 40], ['tanh', [29]

fitness: [0.09664800763130188, 0.10570944100618362, 0.10370893031358719, 0.10167598724365234, 0.07186762243509293, 0.09822839498519897, 0.10389160364866257, 0.09027495235204697, 0.09544934332370758, 0.11089536547660828] 

fitness medio de la población: 0.09783496484160423 

mejor fitness:  0.07186762243509293 

GENERATION 12 

[['tanh', [17, 8, 12, 28], 'adamax', 392, 11], ['tanh', [29], 'adamax', 392, 11], ['tanh', [17, 8, 12, 28], 'adamax', 392, 11], ['tanh', [29], 'adamax', 392, 11], ['tanh', [29], 'adamax', 392, 11], ['tanh', [29], 'adamax', 392, 11], ['tanh', [29], 'adamax', 392, 11], ['tanh', [29], 'adamax', 392, 11], ['tanh', [17, 8, 12, 28], 'adamax', 392, 11], ['tanh', [29], 'adamax', 392, 11]] 

fitness: [0.09800246357917786, 0.10822021216154099, 0.08527354896068573, 9.641816139221191, 0.10835427045822144, 9.641816139221191, 0.10685007274150848, 0.10356612503528595, 0.10193414986133575, 0.09194961190223694] 

fitness medio de la población: 2.0087782733142374 

mejor fitness: 

En este caso parece que el mejor resultado obtenido se ha dado en la última generación o iteración. No obstante no siempre es así. Debido a cierto componente de aleatoriedad debido a cruces y mutaciones, algunas buenas soluciones o buenos hiperparámetros pueden perderse, ocasionando que a veces que la mejor solución en un proceso de optimización en concreto no se encuentra en las generaciones finales, sino en generaciones anteriores, ya que ha habido cierta deriva que ha ocasionado poblaciones con individuos con peores fitness que sus predecesores. Por ello es conveniente tener un historial como el diccionario que en este notebook se ha creado, para poder registrar las distintas soluciones y poder rescatar una buena solución que permita resolver el problema o indicarnos hacia dónde orientar la solución del problema.